# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


C:\Users\Winson\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print (df.head())

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [3]:
# frequency table for prestige and whether or not someone was admitted

pd.crosstab(df.admit,df.prestige)

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,28,95,93,55
1,33,53,28,12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [4]:
dummies = pd.get_dummies(df['prestige'])
print (dummies)

     1.0  2.0  3.0  4.0
0      0    0    1    0
1      0    0    1    0
2      1    0    0    0
3      0    0    0    1
4      0    0    0    1
5      0    1    0    0
6      1    0    0    0
7      0    1    0    0
8      0    0    1    0
9      0    1    0    0
10     0    0    0    1
11     1    0    0    0
12     1    0    0    0
13     0    1    0    0
14     1    0    0    0
15     0    0    1    0
16     0    0    0    1
17     0    0    1    0
18     0    1    0    0
19     1    0    0    0
20     0    0    1    0
21     0    1    0    0
22     0    0    0    1
23     0    0    0    1
24     0    1    0    0
25     1    0    0    0
26     1    0    0    0
27     0    0    0    1
28     0    1    0    0
29     1    0    0    0
..   ...  ...  ...  ...
370    0    1    0    0
371    0    0    1    0
372    1    0    0    0
373    1    0    0    0
374    0    1    0    0
375    0    0    0    1
376    0    1    0    0
377    0    1    0    0
378    0    0    1    0
379    0    1   

#### 2.2 When modeling our class variables, how many do we need? 



Answer: 3

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [5]:
dummy_ranks = pd.get_dummies(df['prestige'], prefix='prestige')

cols_to_keep = ['admit', 'gre', 'gpa']

data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
print (data.head())

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             1             0
1      1  660.0  3.67             0             1             0
2      1  800.0  4.00             0             0             0
3      1  640.0  3.19             0             0             1
4      0  520.0  2.93             0             0             1


C:\Users\Winson\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """


In [6]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted

pd.crosstab(df['admit'], df['prestige'], rownames=['admit'])


prestige,1.0,2.0,3.0,4.0
admit,,,,
0,28,95,93,55
1,33,53,28,12


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [7]:
p1 = 33/(28+33)
odds1 = p1/(1-p1)
odds1

1.1785714285714288

In [8]:
#log odds 

lo1 = np.log(odds1)
lo1

0.16430305129127654

In [9]:
# odds ratio
np.exp(lo1)

1.1785714285714288

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [10]:
p2 = (93)/(243+93)
odds2 = p2/(1-p2)
odds2

0.3827160493827161

In [11]:
#log odds 

lo2 = np.log(odds2)
lo2

-0.9604619501872924

#### 3.3 Calculate the odds ratio

In [12]:
#odds ratio if you did not attend #1 ranked school
or2 = np.exp(lo2) 
or2

0.3827160493827161

#### 3.4 Write this finding in a sentenance: 

Answer: you have a 38% chance of been admitted into the universtiy if you attended a #1 ranked college. 

#### 3.5 Print the cross tab for prestige_4

In [13]:
pd.crosstab(data['admit'], data['prestige_4.0'])

prestige_4.0,0,1
admit,,
0,216,55
1,114,12


#### 3.6 Calculate the OR 

In [14]:
p4 = 12/(12+55)
logodds4 = np.log(p4/(1-p4))
logodds4

-1.5224265354444706

In [15]:
or4 = np.exp(logodds4)
or4

0.21818181818181817

#### 3.7 Write this finding in a sentence

Answer: you have a 21% chance of been admitted to a university if you attended a #4 ranked college. 

## Part 4. Analysis

In [16]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
print (data.head())

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             1             0
1      1  660.0  3.67             0             1             0
2      1  800.0  4.00             0             0             0
3      1  640.0  3.19             0             0             1
4      0  520.0  2.93             0             0             1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [17]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [18]:
#column used to train the model - admit has be removed by[1:]

train_cols = data.columns[1:]
train_cols

Index(['gre', 'gpa', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0',
       'intercept'],
      dtype='object')

#### 4.2 Fit the model

In [19]:
# fit the model
logit = sm.Logit(data['admit'], data[train_cols])
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [20]:
print (result.summary2())

                        Results: Logit
Model:              Logit            No. Iterations:   6.0000  
Dependent Variable: admit            Pseudo R-squared: 0.082   
Date:               2018-07-11 17:40 AIC:              467.6399
No. Observations:   397              BIC:              491.5435
Df Model:           5                Log-Likelihood:   -227.82 
Df Residuals:       391              LL-Null:          -248.08 
Converged:          1.0000           Scale:            1.0000  
---------------------------------------------------------------
                 Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
---------------------------------------------------------------
gre              0.0022   0.0011  2.0280 0.0426  0.0001  0.0044
gpa              0.7793   0.3325  2.3438 0.0191  0.1276  1.4311
prestige_2.0    -0.6801   0.3169 -2.1459 0.0319 -1.3013 -0.0589
prestige_3.0    -1.3387   0.3449 -3.8819 0.0001 -2.0146 -0.6628
prestige_4.0    -1.5534   0.4175 -3.7211 0.0002 -2.3716 -0.7352
i

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [26]:
print (np.exp(result.params))

gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
intercept       0.020716
dtype: float64


In [27]:
# CI intervals
conf = result.conf_int()
conf['OR'] = result.params
conf.columns = ['2.5%', '97.5%', 'OR']
print (np.exp(conf))


                  2.5%     97.5%        OR
gre           1.000074  1.004372  1.002221
gpa           1.136120  4.183113  2.180027
prestige_2.0  0.272168  0.942767  0.506548
prestige_3.0  0.133377  0.515419  0.262192
prestige_4.0  0.093329  0.479411  0.211525
intercept     0.002207  0.194440  0.020716


#### 4.5 Interpret the OR of Prestige_2

Answer: your chances of been admitted to universtity will reduce by 50% if you attended a college ranked #2 compared to someone who attended a college ranked #1 

#### 4.6 Interpret the OR of GPA

Answer: for every unit increase in GPA score,the odds of been admitted to university will increase by 2.18 if you went to a #1 ranked college. 

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [38]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(df['gre'].min(), df['gre'].max(), 10)
print (gres)
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(df['gpa'].min(), df['gpa'].max(), 10)
print (gpas)
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
import itertools
l1 = list(itertools.product(gres, gpas, [1,2,3,4], [1.0]))
df_all = pd.DataFrame(l1, columns = ['gre', 'gpa', 'pres', 'int'])
df_all.head()

[220.         284.44444444 348.88888889 413.33333333 477.77777778
 542.22222222 606.66666667 671.11111111 735.55555556 800.        ]
[2.26       2.45333333 2.64666667 2.84       3.03333333 3.22666667
 3.42       3.61333333 3.80666667 4.        ]


,gre,gpa,pres,int
0,220.0,2.260000,1,1.0
1,220.0,2.260000,2,1.0
2,220.0,2.260000,3,1.0
3,220.0,2.260000,4,1.0
4,220.0,2.453333,1,1.0


#### 5.1 Recreate the dummy variables

In [54]:
# recreate the dummy variables
d = pd.get_dummies(df_all['pres'], prefix = 'pres')
d.drop(labels = ['pres_1'], axis = 1, inplace = True)

df_mat = df_all.join(d).drop(labels = ['pres', 'int'], axis = 1)

x_new = sm.add_constant(df_mat, prepend = False)
x_new.head()


,gre,gpa,pres_2,pres_3,pres_4,const
0,220.0,2.260000,0,0,0,1.0
1,220.0,2.260000,1,0,0,1.0
2,220.0,2.260000,0,1,0,1.0
3,220.0,2.260000,0,0,1,1.0
4,220.0,2.453333,0,0,0,1.0


#### 5.2 Make predictions on the enumerated dataset

In [47]:
x_new['admit_pred'] = result.predict(x_new)

print (x_new.head())

     gre       gpa  pres_2  pres_3  pres_4  const  admit_pred
0  220.0  2.260000       0       0       0    1.0    0.164173
1  220.0  2.260000       1       0       0    1.0    0.090492
2  220.0  2.260000       0       1       0    1.0    0.048977
3  220.0  2.260000       0       0       1    1.0    0.039890
4  220.0  2.453333       0       0       0    1.0    0.185907


In [48]:
x_new.tail(4)

,gre,gpa,pres_2,pres_3,pres_4,const,admit_pred
396,800.0,4.0,0,0,0,1.0,0.734040
397,800.0,4.0,1,0,0,1.0,0.582995
398,800.0,4.0,0,1,0,1.0,0.419833
399,800.0,4.0,0,0,1,1.0,0.368608


#### 5.3 Interpret findings for the last 4 observations

Answer: if GPA and GRE is equal the probablity of being admitted to university is higher if you went to a #1 and #2 college whereas with the same GPA and GRE score if you went to a #3 or #4 ranked college you're proababilyt of being admitted is likely to be lower. 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.

In [55]:
 def isolate_and_plot(variable):
      # isolate gre and class rank
      grouped = pd.pivot_table(x_new, values=['admit_pred'], index=[variable, 'pres'],
                  aggfunc=np.mean)
      # make a plot
      colors = 'rbgyrbgy'
      for col in combos.prestige.unique():
          plt_data = grouped.ix[grouped.index.get_level_values(1)==col]
          pl.plot(plt_data.index.get_level_values(0), plt_data['admit_pred'], color=colors[int(col)])

      pl.xlabel(variable)
      pl.ylabel("P(admit=1)")
      pl.legend(['1', '2', '3', '4'], loc='upper left', title='Prestige')
      pl.title("Prob(admit=1) isolating " + variable + " and presitge")
      pl.show()

isolate_and_plot('gre')
isolate_and_plot('gpa')

KeyError: 'admit_pred'